In [28]:
from utils.model import EFLContrastiveLearningModel
import os
import torch
from types import SimpleNamespace
from transformers import BertTokenizer



args = SimpleNamespace(
                        model_name_or_path='./model/checkpoint-2000000',
                        vocab_path='/home/tmax/kibong_choi/EFL_ContrastiveLearning/tokenizer/version_1.9',
                        batch_size=4,
                        method='efl_scl',
                        pooler_option='cls'
                        )
tokenizer = tokenizer = BertTokenizer.from_pretrained(args.vocab_path,
                                                do_lower_case=False,
                                                unk_token='<unk>',
                                                sep_token='</s>',
                                                pad_token='<pad>',
                                                cls_token='<s>',
                                                mask_token='<mask>',
                                                model_max_length=256)

model_path = 'model/saved_model/STEP_200_efl_scl_TASKsentiment_LR5e-05_WD0.1_LAMBDA0.9_POOLERcls_TEMP0.25'
model = EFLContrastiveLearningModel(args)
model_state_dict = torch.load(os.path.join(model_path, 'model_state_dict.pt'))    
model.load_state_dict(model_state_dict)
model.eval()
    

file /home/tmax/kibong_choi/EFL_ContrastiveLearning/tokenizer/version_1.9/config.json not found
Some weights of the model checkpoint at ./model/checkpoint-2000000 were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


EFLContrastiveLearningModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [8]:
from konlpy.tag import Mecab

mecab = Mecab()

In [54]:
# s1 = '그럼 제가 상식적으로 여태까지 살아오면서 에이에스를 요청을 하면은 어 일 주일 안에는 다 오시거나 뭐 기사님이 뭐 시간을 맞춰서 연락을 주시거나 하는데니까 여기는 기사님이 오시는 것도 한 달이 걸리고 일을 몰아서 오시기 때문에 장당할 수도 없는 거고 기사님이 오셔가지고 이거 수뢰하는 것도 시간이 걸리잖아요'
s1 = '그 샤오미 에이 피컨데 그게 본체랑 케이를 그 같이 파는 거를 샀는데 페이지가 안 왔어요'
s2 = '이것은 부정 문장입니다.'
s3 = '이것은 긍정 문장입니다.'



In [55]:
s1 = mecab.morphs(s1)
s1 = " ".join(s1)
s2 = mecab.morphs(s2)
s2 = " ".join(s2)
s3 = mecab.morphs(s3)
s3 = " ".join(s3)
texts = [(s1, s2), (s1, s3)]

batch = tokenizer(texts,
                  padding=True,
                  return_token_type_ids=False,
                  return_tensors='pt'
                  )

output = model(**batch)
logits = output['logits']
preds = torch.argmax(logits[:,1])

result = '부정' if preds.item() == 0 else '긍정'

print(s1)
print(s2)
print(f'NOT ENTAIL : {logits[0][0]}')
print(f'ENTAIL : {logits[0][1]}')
print(s3)
print(f'NOT ENTAIL : {logits[1][0]}')
print(f'ENTAIL : {logits[1][1]}')
print(f'RESULT : {result}')

그 샤오미 에이 피 컨데 그게 본체 랑 케이 를 그 같이 파 는 거 를 샀 는데 페이지 가 안 왔어요
이것 은 부정 문장 입니다 .
NOT ENTAIL : -0.6898468136787415
ENTAIL : 0.12583734095096588
이것 은 긍정 문장 입니다 .
NOT ENTAIL : 0.7833032011985779
ENTAIL : -0.7389132380485535
RESULT : 부정
